In [2]:
import os
import random
import cv2
import torch
import pandas as pd
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
import numpy as np
import matplotlib
matplotlib.use('Agg')  # Non-interactive backend

# === Force CPU to avoid CUDA errors ===
device = torch.device("cpu")

# === Emotion classes and genre mapping for mood improvement ===
classes = ['anger', 'disgust', 'fear', 'happiness', 'neutral', 'sadness', 'surprise']
emotion_to_genres = {
    'sadness': ['Comedy', 'Musical', 'Animation', 'Romance'],
    'fear': ['Comedy', 'Animation', 'Musical', 'Children'],
    'surprise': ['Fantasy', 'Sci-Fi', 'Action', 'Adventure'],
    'happiness': ['Adventure', 'Sci-Fi', 'Fantasy', 'Action'],
    'disgust': ['Romance', 'Animation', 'Adventure', 'Drama'],
    'anger': ['Adventure', 'Fantasy', 'Animation', 'Comedy', 'Action'],
    'neutral': ['Drama', 'Documentary', 'Mystery', 'Crime', 'Thriller']
}

# === Load model ===
model = efficientnet_b0(weights=EfficientNet_B0_Weights.IMAGENET1K_V1)
model.classifier[1] = torch.nn.Sequential(
    torch.nn.Dropout(0.7),
    torch.nn.Linear(model.classifier[1].in_features, len(classes))
)
model.load_state_dict(torch.load("best_model.pth", map_location=device))
model = model.to(device)
model.eval()

# === Load MovieLens data ===
ratings = pd.read_csv("./ml-100k/u.data", sep="\t", header=None,
                      names=["user_id", "movie_id", "rating", "timestamp"])
movies = pd.read_csv("./ml-100k/u.item", sep="|", header=None, encoding="latin-1", usecols=range(24),
                     names=["movie_id", "title", "release_date", "video_release_date", "IMDb_URL",
                            "unknown", "Action", 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime',
                            'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery',
                            'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'])

# === Automatically select a random image ===
image_extensions = ['.png', '.jpg', '.jpeg']
image_files = []
for root, _, files in os.walk("./"):
    for file in files:
        if os.path.splitext(file)[1].lower() in image_extensions:
            image_files.append(os.path.join(root, file))

if not image_files:
    raise FileNotFoundError("❌ No image files found in subfolders.")

image_path = random.choice(image_files)

# === Randomly assign a user ID ===
valid_user_ids = ratings['user_id'].unique()
assigned_user_id = random.choice(valid_user_ids)

# === Image preprocessing ===
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
    ])
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"❌ Error reading image file: {image_path}")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = Image.fromarray(image)
    return transform(image).unsqueeze(0).to(device)

# === Predict emotion ===
def predict_emotion(image_path):
    image = preprocess_image(image_path)
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
    return classes[predicted.item()]

# === Extract actual class from folder name ===
def get_actual_class(image_path):
    folder_name = os.path.basename(os.path.dirname(image_path)).lower()
    return folder_name if folder_name in classes else "Unknown"

# === Popularity-based recommendation ===
def popularity_based_recommendations(emotion, top_n=5):
    genres = emotion_to_genres.get(emotion, [])
    if not genres:
        print(f"Warning: No genres defined for emotion {emotion}. Using all genres as fallback.")
        genres = movies.columns[6:]
    mask = movies[genres].sum(axis=1) > 0
    filtered = movies[mask].copy()
    avg_ratings = ratings.groupby('movie_id')['rating'].mean().reset_index()
    filtered = filtered.merge(avg_ratings, on='movie_id', how='left')
    return filtered.sort_values('rating', ascending=False)[['title', 'rating'] + genres].head(top_n)

# === Content-based recommendation ===
def content_based_known_user(user_id, emotion, top_n=5):
    genres = emotion_to_genres.get(emotion, [])
    if not genres:
        print(f"Warning: No genres defined for emotion {emotion}. Using all genres as fallback.")
        genres = movies.columns[6:]
    
    user_ratings = ratings[ratings['user_id'] == user_id][['movie_id', 'rating']]
    high_rated = user_ratings[user_ratings['rating'] >= 4][['movie_id']]
    user_movies = high_rated.merge(movies, on='movie_id', how='inner')
    
    genre_cols = movies.columns[6:]
    
    if len(user_movies) == 0 or user_movies[genres].sum().sum() == 0:
        print(f"Warning: User {user_id} has no high-rated movies in {genres}. Using popularity-based fallback.")
        return popularity_based_recommendations(emotion, top_n)
    
    user_profile = user_movies[genre_cols].sum() / len(user_movies)
    
    mask = movies[genres].sum(axis=1) > 0
    candidate_movies = movies[mask].copy()
    
    scores = candidate_movies[genre_cols].dot(user_profile)
    candidate_movies['score'] = scores
    avg_ratings = ratings.groupby('movie_id')['rating'].mean().reset_index()
    candidate_movies = candidate_movies.merge(avg_ratings, on='movie_id', how='left')
    
    recommendations = candidate_movies.sort_values(['score', 'rating'], ascending=False)[['title', 'score', 'rating'] + genres].head(top_n)
    return recommendations

# === Collaborative filtering ===
def collaborative_filtering(user_id, emotion, top_n=5):
    genres = emotion_to_genres.get(emotion, [])
    if not genres:
        print(f"Warning: No genres defined for emotion {emotion}. Using all genres as fallback.")
        genres = movies.columns[6:]
    mask = movies[genres].sum(axis=1) > 0
    filtered_ids = movies[mask]['movie_id'].values
    reader = Reader(rating_scale=(1, 5))
    data = Dataset.load_from_df(ratings[['user_id', 'movie_id', 'rating']], reader)
    trainset, _ = train_test_split(data, test_size=0.2, random_state=42)
    algo = SVD()
    algo.fit(trainset)
    
    predictions = [(mid, algo.predict(user_id, mid).est) for mid in filtered_ids]
    predictions.sort(key=lambda x: x[1], reverse=True)
    
    top_preds = []
    for mid, rating in predictions:
        movie_genres = movies[movies['movie_id'] == mid][genres]
        if movie_genres.sum().sum() > 0:
            top_preds.append((mid, rating))
        if len(top_preds) >= top_n:
            break
    
    if len(top_preds) < top_n:
        print(f"Warning: Collaborative filtering found only {len(top_preds)} movies in {genres}. Supplementing with popularity-based.")
        pop_recs = popularity_based_recommendations(emotion, top_n=(top_n - len(top_preds)))
        for _, row in pop_recs.iterrows():
            top_preds.append((movies[movies['title'] == row['title']]['movie_id'].iloc[0], row['rating']))
    
    recommendations = []
    for mid, rating in top_preds[:top_n]:
        movie_info = movies[movies['movie_id'] == mid]
        recommendations.append({
            'movie_id': mid,
            'title': movie_info['title'].iloc[0],
            'rating': rating
        })
        for genre in genres:
            recommendations[-1][genre] = movie_info[genre].iloc[0]
    
    return pd.DataFrame(recommendations)

# === Rate recommendations ===
def rate_recommendations(recommendations, emotion):
    genres = emotion_to_genres.get(emotion, [])
    if not genres:
        print(f"Warning: No genres defined for emotion {emotion}. Using all genres as fallback.")
        genres = movies.columns[6:]
    titles = recommendations['title'].tolist()
    rec_movies = movies[movies['title'].isin(titles)]
    
    matches = 0
    for _, row in rec_movies.iterrows():
        if row[genres].sum() > 0:
            matches += 1
    
    return matches

# === Run full pipeline ===
predicted_emotion = predict_emotion(image_path)
actual_emotion = get_actual_class(image_path)

# Display image
img = cv2.imread(image_path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img_rgb)
plt.axis('off')
plt.title(f"Actual: {actual_emotion.capitalize()} | Predicted: {predicted_emotion.capitalize()}")
plt.show()

print(f"\n📷 Image: {image_path}")
print(f"✅ Actual Emotion: {actual_emotion}")
print(f"🤖 Predicted Emotion: {predicted_emotion}")
print(f"👤 Assigned User ID: {assigned_user_id}")

# Get recommendations
print("\n🎬 Content-Based Recommendations for Assigned User:")
content_recs = content_based_known_user(assigned_user_id, predicted_emotion)
print(content_recs)
content_score = rate_recommendations(content_recs, predicted_emotion)
print(f"Content-Based Recommendation Score: {content_score}/5")

print("\n🎬 Collaborative Filtering Recommendations for Assigned User:")
collab_recs = collaborative_filtering(assigned_user_id, predicted_emotion)
print(collab_recs)
collab_score = rate_recommendations(collab_recs, predicted_emotion)
print(f"Collaborative Filtering Recommendation Score: {collab_score}/5")

print("\n🎬 Popularity-Based Recommendations for New User:")
pop_recs = popularity_based_recommendations(predicted_emotion)
print(pop_recs)
pop_score = rate_recommendations(pop_recs, predicted_emotion)
print(f"Popularity-Based Recommendation Score: {pop_score}/5")

C:\Users\Atena\AppData\Local\Temp\ipykernel_18292\2884929531.py:207: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()



📷 Image: ./CK+\happiness\S050_006_00000023.png
✅ Actual Emotion: happiness
🤖 Predicted Emotion: happiness
👤 Assigned User ID: 226

🎬 Content-Based Recommendations for Assigned User:
                               title     score    rating  Adventure  Sci-Fi  \
54   Empire Strikes Back, The (1980)  1.205882  4.204360          1       1   
242                      Diva (1981)  1.029412  3.727273          0       0   
41      2001: A Space Odyssey (1968)  1.000000  3.969112          0       1   
287                  Best Men (1997)  1.000000  3.400000          0       0   
87           Devil's Own, The (1997)  1.000000  3.108333          0       0   

     Fantasy  Action  
54         0       1  
242        0       1  
41         0       0  
287        0       1  
87         0       1  
Content-Based Recommendation Score: 5/5

🎬 Collaborative Filtering Recommendations for Assigned User:
   movie_id                      title    rating  Adventure  Sci-Fi  Fantasy  \
0        50           